# Question 2.1: Random PCFG Generation

In this question, we implement the function pcfg_generate(grammar) which will generate a random tree from a PCFG according to its generative probability model. In other words, we implement a sample method from the PCFG distribution. 
We use a recursive method: Given a symbol, we randomly choose a production from the that symbol. The base case is a Terminal. 

In [22]:
from nltk import PCFG, ProbabilisticProduction, MLEProbDist, FreqDist
from nltk.grammar import Nonterminal, Production
from nltk import probability
import re
import numpy as np


def getProbDist(productions):
    dict = {}
    for p in productions:
        dict[p.rhs()]=p.prob()
    return probability.DictionaryProbDist(dict)


def get_random_production_helper(nt,g):
    if not isinstance(nt, Nonterminal):
        return ""
    else:
        productions = g.productions(nt)
        p = getProbDist(productions)
        t = p.generate()
        res = ""
        for x in t:
            if isinstance(x,Nonterminal):
                res =  res + " (" + str(x) + " " + get_random_production_helper(x, g) + ")"
            else:
                res = res + str(x)
        return res


def pcfg_generate(g):
    res = get_random_production_helper(g.start(),g)
    return "(S" + res + ")"

Let's test our code with some example grammer:

In [23]:
g = PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.5] | NP PP [0.25] | 'John' [0.1] | 'I' [0.15]
    Det -> 'the' [0.8] | 'my' [0.2]
    N -> 'man' [0.5] | 'telescope' [0.5]
    VP -> VP PP [0.1] | V NP [0.7] | V [0.2]
    V -> 'ate' [0.35] | 'saw' [0.65]
    PP -> P NP [1.0]
    P -> 'with' [0.61] | 'under' [0.39]
    """)

print(pcfg_generate(g))

(S (NP  (Det the) (N man)) (VP  (V ate) (NP  (Det the) (N telescope))))


Now, we want to validate the generate function.

## 2.1.1
We generate 1,000 random trees using nltk.grammar.toy_pcfg2 - and store the resulting trees in a file "toy_pcfg2.gen". 

In [24]:
from nltk.grammar import toy_pcfg2


def generate_corpus():
    f = open("toy_pcfg2.gen", "w+")
    for i in range(1000):
        s = pcfg_generate(toy_pcfg2)
        f.write(s + "\r\n")
    f.close()

generate_corpus()

Let's check some of the generated trees:

In [25]:
file = open("toy_pcfg2.gen", "r")
lines = file.readlines()
for i in range(5):
    if lines[i] != "\n":
        print(lines[i])

(S (NP  (NP  (Det the) (N telescope)) (PP  (P with) (NP  (Name Bob)))) (VP  (V ran) (NP  (Det a) (N cookie))))

(S (NP  (Det a) (N cookie)) (VP  (V ate)))

(S (NP  (Det the) (N telescope)) (VP  (V ate) (NP  (NP  (Det the) (N hill)) (PP  (P under) (NP  (NP  (NP  (Name Bob)) (PP  (P with) (NP  (NP  (Det a) (N boy)) (PP  (P with) (NP  (Det the) (N boy)))))) (PP  (P with) (NP  (Det a) (N hill))))))))



## 2.1.2 
Now, we compute the frequency distribution of each non-terminal and pre-terminal in the generated corpus. We construct one distribution per non-terminal. First, we initialize 2 global dictionaries. one counts the number of times a given lhs occurs, and the other counts the number of times rhs occurs. Then we transform the string to tree, using the method create_tree, scan it, and update the dictionaries. 

In [26]:

def create_tree(sent):
    items = re.findall(r"\(|\)|\w+", sent)
    return tree_helper(1,items)[0]


def tree_helper(index,items):
    result = []
    item = items[index]
    while item != ")":
        if item == "(":
            subtree, index = tree_helper(index + 1,items)
            result.append(subtree)
        else:
            result.append(item)
        index += 1
        item = items[index]
    return result, index




def leaf(t):
    if len(t) != 2:
        return False
    for x in t:
        if isinstance(x, list):
            return False
    return True


def empirical_pcdg_helper(t,lcount,pcount):
    if leaf(t):
        nt = Nonterminal(t[0])
        lcount[nt] = lcount.get(nt,0)+1
        prod = Production(nt, [t[1]])
        pcount[prod] = pcount.get(prod,0) + 1
    else:
        left = Nonterminal(t[0])
        if len(t) == 2:
            right = Nonterminal(t[1][0])
            prod = Production(left,[right])
            lcount[left] = lcount.get(left, 0) + 1
            pcount[prod] = pcount.get(prod, 0) + 1
            empirical_pcdg_helper(t[1],lcount,pcount)
        else:
            right1 = Nonterminal(t[1][0])
            right2 = Nonterminal(t[2][0])
            lcount[left] = lcount.get(left, 0) + 1
            prod = Production(left,[right1,right2])
            pcount[prod] = pcount.get(prod, 0) + 1
            empirical_pcdg_helper(t[1],lcount,pcount)
            empirical_pcdg_helper(t[2], lcount, pcount)


def get_empirical_pcfg():
    global lcount, pcfg
    lcount = {}
    pcount = {}
    file = open("toy_pcfg2.gen", "r")
    lines = file.readlines()
    for s in lines:
        if s != "\n":
            t = create_tree(s)
            empirical_pcdg_helper(t, lcount, pcount)
    prods = [
        ProbabilisticProduction(p.lhs(), p.rhs(), prob=pcount[p] / lcount[p.lhs()])
        for p in pcount]
    pcfg = PCFG(toy_pcfg2.start(), prods)
    return pcfg



def getFreqDist(productions):
    dict = {}
    for p in productions:
        dict[p.rhs()]=p.prob()
    return FreqDist(dict)


pcfg = get_empirical_pcfg()
pcfg_nt = []
for p in pcfg.productions():
    if p.lhs() not in pcfg_nt:
        pcfg_nt.append(p.lhs())
dist_dict = {}
for nt in pcfg_nt:
    nt_prods = pcfg.productions(nt)
    fd = getFreqDist(nt_prods)
    dist_dict[nt] = fd



dist_dict contains a distribution for each Non-Terminal. for example, let's check dist_dict[NP]


In [27]:
dist = dist_dict[Nonterminal("NP")]
print(dist.keys())
print(dist.values())

dict_keys([(NP, PP), (Det, N), (Name,)])
dict_values([0.31912964641885766, 0.40072529465095197, 0.28014505893019037])


If we compare it to the real NP distribution, we can see that it is a good estimation:

    NP   -> NP PP         [.31]

    NP   -> Det N         [.41]
    
    NP   -> Name          [.28]
    
  

## 2.1.3
Now, for each distribution, we want to compute the KL-divergence between the MLE estimation of the probability distribution constructed on the test corpus and toy_pcfg2. The MLE estimation is obtained by applying the MLEProbDist estimator to the observed empirical frequency distribution. The KL-divergence is a measure of how one probability distribution is different from a another distribution, where  KL divergence of 0 indicates that the two distributions are identical. It is given by:

$$
D_\text{KL}(P \parallel Q) = \sum_{x\in\mathcal{X}} P(x) \log\left(\frac{P(x)}{Q(x)}\right)
$$

In [28]:
non_tr = []
for p in toy_pcfg2.productions():
    if not p.lhs() in non_tr:
        non_tr.append(p.lhs())



def get_kl(p, q):
    p = np.asarray(p)
    q = np.asarray(q)

    return np.sum(p * np.log(p / q))



for nt in non_tr:
    mle = MLEProbDist(dist_dict[nt])
    prod_nt = toy_pcfg2.productions(nt)
    t_dist = []
    mle_dist = []
    for pr in prod_nt:
        t_dist.append(pr.prob())
        mle_dist.append(mle.prob(pr.rhs()))
    kl = get_kl(t_dist,mle_dist)
    print(str(nt) + ': ' + str(kl))



S: 0.0
VP: 0.00022778125911425244
NP: 0.0002384032349946833
PP: 0.0
V: 0.002103657939921455
N: 0.0018825024734908987
Name: 1.0280520433252473e-07
P: 0.0009537747233433827
Det: 0.0002967290271206069


## 2.1.4 

According to the results, the MLE estimation of the test corpus is very similar to the distribution. 
Since the toy_pcfg2 is a small grammer (about 30 rules), 1000 trees are good enough. In some cases we got that the KL divergence equals 0, as in the case of the nonterminal "S". This may seem too accurate, but this result make sense, since the true grammer toy_pcfg2 contains only one production for S: 
S    -> NP VP [1.0]
